In [1]:
# Gregor von Laszewski laszewski@gmail.com

In [2]:

from cloudmesh.common.util import readfile
from glob import glob
from cloudmesh.common.Shell import Shell
import sys
from io import StringIO
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

files = glob("outputs/*.out")

print (files)

[]


In [3]:
def get(value):
    return float(df[df['timer'] == value]["time"])


data = []

for filename in files:
    print (filename)
    content = readfile(filename)
    lines = Shell.find_lines_from(content, "# inference")
    lines = Shell.find_lines_with(lines, "# csv")
    content = "\n".join(lines)
    try:
        #print (content)
        # df = pd.read_csv(content)
        df = pd.read_csv(StringIO(content), sep=",")
        df = df.drop(columns=["# csv", "status", "sum", "start", "uname.node", "user",
                              "uname.system", "platform.version", "msg", "tag"])
        for exclude in ["init",'number_of_ranks', "number_of_nodes", "version",
                        "result", "run_stop", "submission_status"]:
            df = df[df['timer'] != exclude]
        # df = df.drop(rows=["init"])

        df['time'] = df['time'].astype(float)

        total = get("total")
        training = get("training block")
        inference = get("inference block")

        #df[["rest"]] = ["rest", 0.0]

        rest = total - training - inference
        rest_row = pd.DataFrame([{'timer':'rest', 'time': rest}])

        df = pd.concat([df, rest_row], ignore_index=True)
        # df = pd.concat([rest_row,df.loc[:]]).reset_index(drop=True)

        df[["job"]] = filename.replace(".out","").replace("outputs/", "")
        df = df.reset_index()
        df = df.drop(columns=["index"])
        #display(df)
        #print (total, training, inference)
        df.reset_index(drop=True, inplace=True)
        data.append(df)
    except:
        print(f"ERROR: {filename}")

In [4]:
data = pd.concat(data, ignore_index=True)
data

ValueError: No objects to concatenate

In [ ]:
def simplify_job(name):
   return " ".join(name.split("-")[:-2]).replace("fox", "special")


def get_by_timer(data, timer):
    d = data[data['timer'] == timer]
    d = d.reset_index()
    d["job"] = d["job"].apply(simplify_job)
    d = d.drop(columns=["index"])
    return d

total = get_by_timer(data, "total")

total

In [ ]:
"""
+-------------------------+----------+---------+
| Name                    | Status   |    Time |
|-------------------------+----------+---------+
| total                   | ok       | 203.461 |
| init                    | ok       |   0     |
| number_of_ranks         | ok       |   0     |
| number_of_nodes         | ok       |   0     |
| version                 | ok       |   0     |
| training block          | ok       | 152.313 |
| training                | ok       | 152.299 |
| loaddata                | ok       |   1.766 |
| training_on_mutiple_GPU | ok       | 148.641 |
| inference block         | ok       |  51.018 |
| inference               | ok       |   0.006 |
| load model              | ok       |   0.196 |
| read inference files    | ok       |   0.009 |
| result                  | ok       |   0     |
| run_stop                | ok       |   0     |
| submission_status       | ok       |   0     |
+-------------------------+----------+---------+
"""

addons = pd.DataFrame(
    [
        {'timer':'total', 'time': 203.461, 'job': 'rtx3090 desktop'}
        {'timer':'total', 'time': 660.296, 'job': 'a100 greene'}
    ])
total = pd.concat([total, addons], ignore_index=True)


total

In [ ]:
total = total.sort_values(by=['time'], ascending=False)

In [ ]:
def save_image(p, name, dpi=300):
    # Shell.mkdir(directory)
    p.figure.savefig(f"{name}.svg", bbox_inches="tight")
    p.figure.savefig(f"{name}.png", bbox_inches="tight", dpi=dpi)
    p.figure.savefig(f"{name}.pdf", bbox_inches="tight")
    p.figure.savefig(f"{name}.tiff", bbox_inches="tight")

In [ ]:
def plot_by_timer(data, timer):
    plt.xticks(rotation=45)
    p = sns.barplot(data, x="job", y="time")
    p.set(xlabel = "GPU", ylabel = "Time in s")
    p.set_title(f"{timer} Runtime Comparision of Rivanna GPUs")
    return p

p = plot_by_timer(total, "total")

In [ ]:
save_image(p, "./images/comparision-epoch-1", dpi=300)

In [ ]:
training = get_by_timer(data, "training block")
plot_by_timer(training, "training block")

training

In [ ]:
inference = get_by_timer(data, "inference block")
plot_by_timer(inference, "inference block")

inference

In [ ]:

_data = data[data['timer'].isin(["total","training block","inference block"])]
_data = _data.sort_values(by=['time'], ascending=False)
_data = _data.reset_index()
_data["job"] = _data["job"].apply(simplify_job)
_data = _data.drop(columns=["index"])

# _data["job"] = _data["job"].apply(simplify_job)
# _data = _data.drop(columns=["index"])

_data

In [ ]:
def get(name):
    content = data[data.timer.isin([name])]['time'].tolist()
    return {name: content}


#plt.xticks(rotation=45)
#p.set(xlabel = "GPU", ylabel = "Time in s")
#p.set_title("Runtime Comparision of Rivanna GPUs")


stacked = pd.DataFrame(
    [get['job'], get('total'), get('training block')]
)

stacked

#stacked.set_index('job').plot(kind='bar', stacked=True, color=['blue', 'red', 'orange'])